# Libraries and Input Parameters

In [1]:
from pylab import *
import numpy as np
import arviz as az
import pandas as pd
import seaborn as sns
import h5py
import ringdown
import matplotlib.pyplot as plt
import scipy
import random
from __future__ import division, print_function
import bilby
from bilby.core.prior import Uniform
from bilby.gw.conversion import convert_to_lal_binary_black_hole_parameters, generate_all_bbh_parameters
from gwpy.timeseries import TimeSeries
import pycbc.catalog
import pycbc.waveform

sns.set_context('notebook')
sns.set_palette('colorblind')

setting __package__ to gwsurrogate.new so relative imports work
__name__ = gwsurrogate.new.spline_evaluation
__package__= gwsurrogate.new
setting __package__ to gwsurrogate.new so relative imports work
setting __package__ to gwsurrogate.new so relative imports work


/nasdata1/ashwin.girish/miniconda3/envs/ringdown/lib/python3.10/site-packages/gwtools/rotations.py:63: UserWarning: Could not import GWFrames, needed for rotations module
  _warnings.warn("Could not import GWFrames, needed for rotations module")
/nasdata1/ashwin.girish/miniconda3/envs/ringdown/lib/python3.10/site-packages/gwtools/__init__.py:11: UserWarning: Could not import rotations, decompositions, or fitfuncs. These are not needed by GWSurrogate.
  _warnings.warn("Could not import rotations, decompositions, or fitfuncs. These are not needed by GWSurrogate.")


In [2]:
#Importing surrogate model
#gwsurrogate.catalog.pull('NRSur7dq4')

In [3]:
#sur = gwsurrogate.LoadSurrogate('NRSur7dq4')

In [4]:
#help(sur)

In [5]:
#Functions used in the code
def norm(arr1,arr2):
    diff = arr1[1] - arr1[0]
    func = scipy.interpolate.interp1d(arr1,arr2)
    nor = 0
    for i in range(0,(len(arr1)-1)):
        diff1 = diff/4
        p1 = func(arr1[i])
        p2 = func(arr1[i]+(diff1))
        p3 = func(arr1[i]+(2*diff1))
        p4 = func(arr1[i]+(3*diff1))
        p5 = func(arr1[i+1])
        nor = nor+((2*diff)/45)*((7*p1)+(32*p2)+(12*p3)+(32*p4)+(7*p5))
    return nor 

def read_strain(file, dname):
    with h5py.File(file, 'r') as f:
        t0 = f['meta/GPSstart'][()]
        T = f['meta/Duration'][()]
        h = f['strain/Strain'][:]
    
        dt = T/len(h)
    
        raw_strain = ringdown.Data(h, index=t0 + dt*arange(len(h)), ifo=dname)
        
        return raw_strain

def next_pow_two(x):
    y = 1
    while y < x:
        y = y << 1
    return y

In [81]:
G1 = pycbc.catalog.catalog.get_source(source='gwtc-1')
E1 = G1['GW150914-v3']
h1_ringdown = 'H-H1_GWOSC_16KHZ_R1-1126259447-32.hdf5'
l1_ringdown = 'L-L1_GWOSC_16KHZ_R1-1126259447-32.hdf5'
right_ascension = 1.95
declination = -1.27
psi_in = 0.82

tgps = E1['GPS']
M_est = E1['final_mass_source']
chi_est = E1['chi_eff']
M1_min = E1['mass_1_source'] + E1['mass_1_source_lower']
M1_max = E1['mass_1_source'] + E1['mass_1_source_upper']
M2_min = E1['mass_2_source'] + E1['mass_2_source_lower']
M2_max = E1['mass_2_source'] + E1['mass_2_source_upper']
ld = E1['luminosity_distance']
ld_l = E1['luminosity_distance'] + E1['luminosity_distance_lower']
ld_u = E1['luminosity_distance'] + E1['luminosity_distance_upper']
CM = E1['chirp_mass_source']
CM_l = CM + E1['chirp_mass_source_lower']
CM_u = CM + E1['chirp_mass_source_upper']

# Bilby

In [82]:
def waveform_model(time,m1,m2,a1x,a1y,a1z,a2x,a2y,a2z,dist):
    G1 = pycbc.catalog.catalog.get_source(source='gwtc-1')
    E1 = G1['GW150914-v3']
    hp,hc = pycbc.waveform.get_td_waveform(approximant = 'NRSur7dq4',
                                           mass1 = m1, mass2 = m2,
                                           spin1x = a1x, spin1y = a1y,
                                           spin1z = a1z, spin2x = a2x,
                                           spin2y = a2y, spin2z = a2z,
                                           distance = dist,
                                           f_lower = 75,
                                           delta_t = 1/4096)
    time = time - E1['GPS']
    #n = int(np.abs(time)/(np.min(np.array(hp.sample_times)))*len(hp.sample_times))
    outp = hp#[n]
    outc = hc#[n]
    return {"plus": outp, "cross": outc}

injection_parameters = dict(
    m1=36.0,
    m2=30.0,
    a1x=0.0,
    a1y=0.0,
    a1z=0.0,
    a2x=0.0,
    a2y=0.0,
    a2z=0.0,
    ra=0,
    dec=0,
    psi=0,
    t0=0.0,
    geocent_time=0.0,
)

In [83]:
logger = bilby.core.utils.logger
outdir = "outdir"
label = "GW150914"

trigger_time = tgps
detectors = ["H1", "L1"]
maximum_frequency = 1024
minimum_frequency = 75
roll_off = 0.4  #Roll off duration of tukey window in seconds, default is 0.4s
duration = 4  #Analysis segment duration
post_trigger_duration = 0  #Time between trigger time and end of segment
end_time = trigger_time + post_trigger_duration
start_time = end_time - duration

In [84]:
ifo_list = bilby.gw.detector.InterferometerList([])
for det in detectors:
    logger.info("Downloading analysis data for ifo {}".format(det))
    ifo = bilby.gw.detector.get_empty_interferometer(det)
    data = TimeSeries.fetch_open_data(det, start_time, end_time)
    ifo.strain_data.set_from_gwpy_timeseries(data)
    ifo.maximum_frequency = maximum_frequency
    ifo.minimum_frequency = minimum_frequency
    ifo_list.append(ifo)

logger.info("Saving data plots to {}".format(outdir))
bilby.core.utils.check_directory_exists_and_if_not_mkdir(outdir)
ifo_list.plot_data(outdir=outdir, label=label)

15:53 bilby INFO    : Downloading analysis data for ifo H1
15:53 bilby INFO    : Downloading analysis data for ifo L1
15:53 bilby INFO    : Saving data plots to outdir
15:53 bilby INFO    : Generating frequency domain strain from given time domain strain.
15:53 bilby INFO    : Applying a tukey window with alpha=0.1, roll off=0.2
15:53 bilby INFO    : Generating frequency domain strain from given time domain strain.
15:53 bilby INFO    : Applying a tukey window with alpha=0.1, roll off=0.2


In [97]:
prior = bilby.gw.prior.PriorDict()
prior["geocent_time"] = bilby.core.prior.Uniform(trigger_time - 0.1, trigger_time + 0.1, name="geocent_time")
prior["m1"] = bilby.core.prior.Uniform(M1_min, M1_max, r"m1")
prior["m2"] = bilby.core.prior.Uniform(M2_min, M2_max, r"m2")
prior["a1x"] = 0.0 #bilby.core.prior.analytical.Uniform(name='a1x', minimum=0, maximum=0.99)
prior["a1y"] = 0.0 #bilby.core.prior.analytical.Uniform(name='a1y', minimum=0, maximum=0.99)
prior["a1z"] = 0.0 #bilby.core.prior.analytical.Uniform(name='a1z', minimum=0, maximum=0.99)
prior["a2x"] = 0.0 #bilby.core.prior.analytical.Uniform(name='a2x', minimum=0, maximum=0.99)
prior["a2y"] = 0.0 #bilby.core.prior.analytical.Uniform(name='a2y', minimum=0, maximum=0.99)
prior["a2z"] = 0.0 #bilby.core.prior.analytical.Uniform(name='a2z', minimum=0, maximum=0.99)
prior["dist"] = ld
prior["ra"] = right_ascension
prior["dec"] = declination
prior["psi"] = psi_in

In [98]:
duration = 1
sampling_frequency = 1024
outdir = "outdir"
label = "time_domain_source_model"

waveform_generator = bilby.gw.WaveformGenerator(duration = duration,
                                                sampling_frequency = sampling_frequency, 
                                                time_domain_source_model = waveform_model,
                                                start_time = start_time)

15:55 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: None
  time_domain_source_model: __main__.waveform_model
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


In [99]:
#import pycbc.waveform as wf
#hp,hc = wf.get_td_waveform(approximant = 'TaylorT4', mass1 = 10, mass2 = 5, f_lower = 75, delta_t = 1/4096)
#import matplotlib.pyplot as plt
#plt.plot(hp.sample_times, hp)
#plt.plot(hc.sample_times, hc)

In [100]:
#import pycbc.waveform as wf
#hp,hc = wf.get_td_waveform(approximant = 'NRSur7dq4', mass1 = 10, mass2 = 5, f_lower = 73, delta_t = 1/4096)
#n = int(0.15/np.min(np.array(hp.sample_times))*len(hp.sample_times))
#print(hp[n])
#import matplotlib.pyplot as plt
#plt.plot(hp.sample_times, hp)
#plt.plot(hc.sample_times, hc)

In [101]:
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list,
    waveform_generator,
    priors=prior,
)

In [102]:
result = bilby.run_sampler(
    likelihood,
    prior,
    sampler="dynesty",
    nlive=1000,
    #npoints=20000,
    check_point_delta_t=600,
    check_point_plot=True,
    injection_parameters=injection_parameters,
    npool=1,
    conversion_function=bilby.gw.conversion.generate_all_bbh_parameters,
)

15:55 bilby INFO    : Running for label 'label', output will be saved to 'outdir'
15:55 bilby INFO    : Using lal version 7.2.2
15:55 bilby INFO    : Using lal git version Branch: None;Tag: lal-v7.2.2;Id: 5168d97ea9fcd06a3b49dd03a1a82d5e700e514e;;Builder: Adam Mercer <adam.mercer@ligo.org>;Repository status: CLEAN: All modifications committed
15:55 bilby INFO    : Using lalsimulation version 4.0.2
15:55 bilby INFO    : Using lalsimulation git version Branch: None;Tag: lalsimulation-v4.0.2;Id: 233cc3963d87688c272b7affe7dd0b962e4c11a0;;Builder: Adam Mercer <adam.mercer@ligo.org>;Repository status: CLEAN: All modifications committed
15:55 bilby INFO    : Search parameters:
15:55 bilby INFO    :   geocent_time = Uniform(minimum=1126259462.3000002, maximum=1126259462.5, name='geocent_time', latex_label='$t_c$', unit=None, boundary=None)
15:55 bilby INFO    :   m1 = Uniform(minimum=32.5, maximum=40.300000000000004, name='m1', latex_label='m1', unit=None, boundary=None)
15:55 bilby INFO    : 

ValueError: operands could not be broadcast together with shapes (103,) (8193,) (103,) 

# Ringdown

In [ ]:
#Input parameters
right_ascension = np.mean(np.array(result.posterior.ra))
declination = np.mean(np.array(result.posterior.dec))
psi_in = np.mean(np.array(result.posterior.psi))

In [ ]:
h_raw_strain = read_strain(h1_ringdown, 'H1')
l_raw_strain = read_strain(l1_ringdown, 'L1')

longest_tau = ringdown.qnms.get_ftau(M_est, chi_est, 0, l=2, m=2)[1]
highest_drate = 1/ringdown.qnms.get_ftau(M_est, chi_est, 1, l=2, m=2)[1]
print('The damping rate of the second tone is: {:.1f} Hz'.format(highest_drate))
print('The time constant of the first tone is: {:.1f} ms'.format(1000*longest_tau))

T = 10*longest_tau
srate = next_pow_two(2*highest_drate)
print('Segment of {:.1f} ms at sample rate {:.0f}'.format(1000*T, srate))

In [ ]:
#Choosing Ringdown model and conditioning data
fit = ringdown.Fit(model='mchi', modes=[(1, -2, 2, 2, 0), (1, -2, 2, 2, 1)])  # use model='ftau' to fit damped sinusoids instead of +/x polarized GW modes
fit.add_data(h_raw_strain)
fit.add_data(l_raw_strain)
fit.set_target(tgps, ra=right_ascension, dec=declination, psi=psi_in, duration=T) #check if ra dec and psi are correct
fit.condition_data(ds=int(round(h_raw_strain.fsamp/srate)), flow=1/T)
fit.compute_acfs()
wd = fit.whiten(fit.analysis_data)

In [ ]:
#Setting Priors and running fit
fit.update_prior(A_scale=5e-21, M_min=35.0, M_max=140.0, flat_A=True)
fit.run(draws=1500,random_seed=1234)

# Calculating Areas

In [ ]:
#Calculating final area 
m = np.array(fit.result.posterior.M)
spin = np.array(fit.result.posterior.chi)
mass = np.array([])
a = np.array([])
for i in range(0,4):
    mass = np.append(mass,m[i])
    a = np.append(a,spin[i])

G = 6.67 * (10**(-11))
c = 9 * (10**9)
area = [0 for i in range(0,len(mass))]
for i in range(0,len(mass)): 
    temp = 8*np.pi*(((G*mass[i])/(c**2))**2)*(1+((1-(a[i]**2))**(1/2)))
    area[i] = temp
area_f = np.array(area)

In [ ]:
#Initial Areas
mass1 = np.array(result.posterior.mass_1_source)
mass2 = np.array(result.posterior.mass_2_source)
a1 = np.array(result.posterior.a_1)
a2 = np.array(result.posterior.a_2)

area1 = np.array([8*np.pi*(((G*mass1[i])/(c**2))**2)*(1+((1-(a1[i]**2))**(1/2))) for i in range(0,len(mass1))])
area2 = np.array([8*np.pi*(((G*mass2[i])/(c**2))**2)*(1+((1-(a2[i]**2))**(1/2))) for i in range(0,len(mass1))])
area_i = np.array([(area1[i] + area2[i]) for i in range(len(area1))]) #total initial area

# Calculating dA/A

In [ ]:
#Gaussian KDES - Check how these look with different smoothening parameters and bandwidths
f2 = scipy.stats.gaussian_kde(area_i,bw_method='silverman') #KDE for initial area
f1 = scipy.stats.gaussian_kde(area_f,bw_method='silverman') #KDE for final area

N = 100 #Number of analysis points
start = np.min(area_i)/2
finish = np.max(area_f)*2
u = np.linspace(start,finish,N)

In [ ]:
n,b,patches = plt.hist(area_f,bins=150)
n = np.append(n,0)
n = n*1e52*1.1 #Arbitrary scaling just for visualising data, no use in further calculations

In [ ]:
plt.plot(u,f1.evaluate(u),label='KDE',color='r')
plt.plot(b,n,label='histogram',color='k')
plt.legend()
plt.title('Final Area')
plt.show()

In [ ]:
n,b,patches = plt.hist(area_i,bins=150)
n = np.append(n,0)
n = n*1e55/2 #Arbitrary scaling just for visualising data, no use in further calculations

In [ ]:
plt.plot(u,f2.evaluate(u),label='KDE',color='r')
plt.plot(b,n,label='histogram',color='k')
plt.legend()
plt.title('Initial Area')
plt.xlim(0.6e-55,1e-55)
plt.show()

In [ ]:
#Integral for A_f/A_i ratio
s = 0.0
f = np.max(area_f)/np.min(area_i) 
I = [0.0 for i in range(0,N)]
z = np.linspace(s,f,N)
x = u
h = (u[1]-u[0])/len(x)

for i in range(0,len(z)):
    for j in range(0,(len(x)-1)):
        h1 = h/4
        k1 = f1.evaluate(x[j]*z[i]) * f2.evaluate(x[j]) * x[j]
        k2 = f1.evaluate((x[j]+(h1))*z[i]) * f2.evaluate(x[j]+(h1)) * (x[j]+(h1))
        k3 = f1.evaluate((x[j]+(2*h1))*z[i]) * f2.evaluate(x[j]+(2*h1)) * (x[j]+(2*h1))
        k4 = f1.evaluate((x[j]+(3*h1))*z[i]) * f2.evaluate(x[j]+(3*h1)) * (x[j]+(3*h1))
        k5 = f1.evaluate(x[j+1]*z[i]) * f2.evaluate(x[j+1]) * x[j+1]
        I[i] = I[i] + ((2*h)/45)*((7*k1)+(32*k2)+(12*k3)+(32*k4)+(7*k5))

In [ ]:
z = z - 1
# Normalising dA/A distribution
temp = np.array([])
for i in I:
    temp = np.append(temp,i)
I = temp
normal = norm(z,I)
I = I/normal

#Plotting dA/A and exporting data to a file
plt.plot(z,I)
plt.show()